<h1><span style="color:red">Re-coding ordinal variables</span></h1>

Using this notbook, you can select ordinal variables and prepend numbers to values. You will have an option to process a survey file received from the current SuAVE application, or import a local CSV file. Then the notebook will let you create a new SuAVE survey with the updated survey file. 

Author: Iakov Vasilyev


## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

## 2. Import libraries

In [ ]:
from __future__ import print_function
import ipywidgets as widgets
import pandas as pd
from IPython.display import Markdown, display
import panel as pn
import re

pn.extension()

def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"
url_partitioned = full_notebook_url.partition('/operations')
base_url = url_partitioned[0];

In [ ]:
def slider(data):
    """
    slider creates an interactive display of a
    data frame.
    
    :param df: data frame
    :returns: interactive dataframe
    """
    
    ## Row Selector widget
    row_selection = pn.widgets.IntSlider(name='Navigate Rows', width=350, 
                                         margin=(0,50,-15,0), end=len(df)-1)

    # Column Selector widget
    col_selection = pn.widgets.IntSlider(name='Navigate Columns', width=350, 
                                         margin=(0,0,5,0), end=len(df.columns))
    
    @pn.depends(row_selection.param.value, col_selection.param.value)
    def navigate_data(row=0, col=0):
        return data.iloc[row:row+5, col:col+10]
    
    sliders = pn.Row(row_selection, col_selection, margin=(0,0,0,10))
    full_widget = pn.Column(sliders, navigate_data)
    return full_widget

def extract_data(path):
    """
    extract_data reads files from various formats
    
    :param link: string representing path to file
    :returns: data frame of file
    """

    # Reading file at path
    if path.endswith(('.txt', 'tsv')):
        try:
            data = pd.read_csv(path, sep='\t', encoding="latin-1")
        except UnicodeDecodeError:
            data = pd.read_csv(path, sep='\t', encoding="ISO-8859-1")
    elif path.endswith('.csv'):
        try:
            data = pd.read_csv(path, encoding="latin-1")
        except UnicodeDecodeError:
            data = pd.read_csv(path, encoding="ISO-8859-1")
    else:
        return None
    
    return data

## 3. Select a survey file from SuAVE or import a local CSV file

In [ ]:
data_select = pn.widgets.RadioBoxGroup(name='Select notebook', options=['Load survey file from SuAVE', 
                                                                        'Import a local CSV file'], 
                                       inline=False)
data_select

In [ ]:
data_input = pn.widgets.FileInput()
    
def check_selection():
    if data_select.value == 'Load survey file from SuAVE':
        global fname
        fname = absolutePath + csv_file
        printmd("<b><span style='color:red'>SuAVE survey will be loaded. Continue to step 4.</span></b>")

    else:
        message = pn.pane.HTML("<b><span style='color:red'>Upload data and continue to step 4.</span></b>")
        return pn.Column(message, data_input)
    
check_selection()

## 4. Visualize the data and assign codes for values of ordinal variables

In [ ]:
if not pd.isnull(data_input.filename):
    fname = absolutePath + data_input.filename
    data_input.save(fname)

# df = extract_data(fname).fillna('')
df = extract_data(fname)

slider(df)

In [ ]:
# defining the panel
            
likcols = df.columns.tolist()
# remove any variable names with qualifiers
likcols = [x for x in likcols if '#' not in x]
# remove variables that have less than 3 or more than 10 unique values
likcols = [x for x in likcols if df[x].nunique() > 2 and df[x].nunique() < 11]
    
if 'updated_df' in globals():
    global updated_df
    updated_df = df.copy()
        
counter = 0
run = False
var_values = []
num_inputs = []
input_rows = []

# Column Selector widget
col_select = pn.widgets.IntSlider(name='Navigate Columns', end=len(df.columns))
    
# Variable Selector widget
var_select = pn.widgets.Select(options=likcols, value="Select Column", width=250)
    
# Update Data Frame widget
updater = pn.widgets.Toggle(name='Apply Codes', button_type='primary', width = 250)
    
# Dictionary to keep track of reserved column names use
violated = {}
remap_text = pn.pane.Markdown('#### Value re-coding: ', width=950)
@pn.depends(updater.param.value, col_select.param.value)
def update_trigger(update, col=0):
    """
    update_trigger updates the existing data frame
    when there are changes to the update widget or
    the column selector widget.
    
    :param update: bool indicated click on update widget
    :param col: integer representing column to display
    :returns: updated data frame
    """
    
    # Please note: Any function that is dependent on a widgets value 
    # (@pn.depends), like this one, will run every time one of the widget's
    # value is changed. However, this is not limited to a user changing the
    # widgets, but also changed programmatically as well. This is done a
    # few times across this function which leads to redundant calls.
    # These redundant calls can cause problems as they can reassign variables.
    # Hence, you may see some extra variables to detect these redundant calls.

    upd = updater.value
    updater.value = False

    # Checks for previous runs of this function
    if 'updated_df' not in globals():
        global updated_df
        updated_df = df.copy()
       
    if var_select.value != "Select Column":
        selected = var_select.value
        uniks = [i for i in updated_df[selected].unique() if not pd.isna(i)]
        mapping = {}
        
        if re.match(r"\d+:",uniks[0]):
            for i in range(len(uniks)):
                if (str(num_inputs[i].value) == ''):
                    mapping[uniks[i]] = uniks[i]
                else:
                    mapping[uniks[i]] = str(num_inputs[i].value) + ": " + uniks[i][uniks[i].find(":")+2:]
        else:
            for i in range(len(var_values)):
                if (str(num_inputs[i].value) == ''):
                    mapping[var_values[i].value] = var_values[i].value
                else:
                    mapping[var_values[i].value] = str(num_inputs[i].value) + ": " + var_values[i].value
                    

        updated = updated_df[selected].apply(lambda var: var if pd.isna(var) else mapping[var])

        updated_df[selected] = updated
        if upd == False:
            remap_text.object = remap_text.object + '\n\n' + "## " + selected
            for i in list(mapping.keys()):
                new_mapping = i + ": " + mapping[i][:mapping[i].find(":")]
                remap_text.object = remap_text.object + '\n' + new_mapping
        
    return updated_df.iloc[:10, col:col+10]

@pn.depends(var_select.param.value)
def variable_rename(var):
    """
    variable_rename renames variables in a data frame
    when the user does so through a text input widget.
    
    :param var: string, variable to be renamed
    :param name: string, new variable name
    :returns: variable rename widget
    """
    #print(var_select.value)
    
    if var_select.value != "Select Column":
        inituniques = [i for i in updated_df[var_select.value].unique() if not pd.isna(i)]
        var_values.clear()
        num_inputs.clear()
        for i in range(len(inituniques)):
            var_values.append(pn.widgets.StaticText(value=str(inituniques[i]), margin=(7,0,5,10)))
            num_inputs.append(pn.widgets.TextInput(value=str(i+1), width=50, margin=(0,23,0,20)))
        #remap_text.object = remap_text.object + '\n' + str(var_values)
        
        var_col = pn.Column(var_values[0])
        num_col = pn.Column(num_inputs[0])
        input_rows = [pn.Row(var_values[i], num_inputs[i]) for i in range(len(var_values))]
        for i in range(1,len(inituniques)):
            var_col.append(var_values[i])    
            num_col.append(num_inputs[i])
        
        input_row = pn.Row(var_col, num_col, margin=(0,0,70,70))
        return input_row
    else:
        return pn.widgets.StaticText(value="Select Column")

In [ ]:
    lefted = pn.Column(var_select, updater, css_classes=['widget-box'])
    full_edit = pn.Row(lefted, variable_rename, margin=(20,50,0,0),css_classes=['widget-box'])
    full_widget = pn.Column(col_select, update_trigger, full_edit, remap_text)
    full_widget

## 5. Generate a new survey and open it in SuAVE

In [ ]:
# new filename

if data_select.value == 'Import a local CSV file':
    csv_file = data_input.filename
df = updated_df
new_file = absolutePath + csv_file[:-4]+'_v1.csv'
printmd("<b><span style='color:red'>A new temporary file will be created at: </span></b>")
print(new_file)
df.to_csv(new_file, index=None)

In [ ]:
#Input survey name

from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)

In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

In [ ]:
referer = survey_url.split("/main")[0] +"/"
upload_url = referer + "uploadCSV"
new_survey_url_base = survey_url.split(user)[0]

import requests
import re
csv = {"file": open(new_file, "rb")}

if data_select.value == 'Import a local CSV file':
    dzc_file = ''
    views = '1110001'
    view='grid'

upload_data = {
    'name': input_text.value,
    'dzc': dzc_file,
    'user':user
}
headers = {
    'User-Agent': 'suave user agent',
    'referer': referer
}

r = requests.post(upload_url, files=csv, data=upload_data, headers=headers)

if r.status_code == 200:
    printmd("<b><span style='color:red'>New survey created successfully</span></b>")
    regex = re.compile('[^0-9a-zA-Z_]')
    s_url = survey_name
    s_url =  regex.sub('_', s_url)

    url = new_survey_url_base + user + "_" + s_url + ".csv" + "&views=" + views + "&view=" + view
    print(url)
    printmd("<b><span style='color:red'>Click the URL to open the new survey</span></b>")
else:
    printmd("<b><span style='color:red'>Error creating new survey. Check if a survey with this name already exists.</span></b>")
    printmd("<b><span style='color:red'>Reason: </span></b>"+ str(r.status_code) + " " + r.reason)